In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfr

In [3]:
! pip install tensorflow_recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.7 MB/s 
     |████████████████████████████████| 511.7 MB 5.3 kB/s 
     |████████████████████████████████| 5.8 MB 34.3 MB/s 
     |████████████████████████████████| 1.6 MB 47.1 MB/s 
     |████████████████████████████████| 438 kB 58.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninsta

In [31]:
ratings = tfds.load("movielens/100k-ratings",split="train")
movies = tfds.load("movielens/100k-movies",split="train")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteVF3R8O/movielens-train.tfrecord*...:   0…

Dataset movielens downloaded and prepared to ~/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [32]:
## taking 2 features id and movie title from ratings

ratings = ratings.map(lambda x:{
    "movie_title":x["movie_title"],
    "user_id":x["user_id"]
})
movies = movies.map(lambda x:x["movie_title"])

In [33]:
class TwoTowerMovielensModel(tfr.Model):
  ### prediction model
  def __init__(self):
    super().__init__()
    embedding_dim = 32
    num_unique_users = 1000
    num_unique_movies = 1700
    eval_batch_size = 128
    ### transforming raw data to numerical representation
    self.user_model = tf.keras.Sequential([
        #turn the raw user ids into contagious interger
        tf.keras.layers.experimental.preprocessing.StringLookup(
            max_tokens=num_unique_users
        ),
        tf.keras.layers.Embedding(num_unique_users,embedding_dim)
    ])
    ## transating movies titles into embeddings
    self.movie_model = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(max_tokens=num_unique_movies),
        tf.keras.layers.Embedding(num_unique_movies, embedding_dim)])
    ## determining which task the model is going to perform..retrieval,ranking or post ranking
    self.task = tfr.tasks.Retrieval(
        metrics=tfr.metrics.FactorizedTopK(candidates=movies.batch(eval_batch_size).map(self.movie_model))
    )

  def compute_loss(self,features,training = False):
    user_embeddings = self.user_model(features['user_id'])
    movies_embeddings = self.movie_model(features["movie_title"])
    return self.task(user_embeddings,movies_embeddings)


In [34]:
model = TwoTowerMovielensModel()
model.compile(optimizer = tf.keras.optimizers.Adagrad(0.1))

In [37]:
model.fit(ratings.batch(4096),verbose=False)

FailedPreconditionError: ignored